In [1]:
import os

In [2]:
%pwd

'd:\\recommendation-engine\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\recommendation-engine'

In [5]:
import pandas as pd

In [6]:
bookings = pd.read_csv("artifacts/data_ingestion/generated_data/bookings.csv")
comments = pd.read_csv("artifacts/data_ingestion/generated_data/comments.csv")
likes = pd.read_csv("artifacts/data_ingestion/generated_data/likes.csv")
bookings.head()
comments.head()
likes.head()

,user_id,liked_event_id,liked_event_organizer_id,like_id
0,beca0aa6-bebd-4c34-9c5b-03edf7b71dc5,f9a6a40e-3b6b-4954-b45a-1c077358587b,60c5d29a-cd56-4a6d-83d8-8417f7dd8c45,3987e583-1fe8-4ec2-a973-914b2b497848
1,2b655047-c558-4fd6-a952-8e61969550e3,b81ab92e-0e6e-4b9b-ad50-6ac361b85a3f,dd0b548a-76c5-4bd0-8527-87ee47c7ae60,1b4c0bfb-f886-4e54-b24d-9b3dcea0e36f
2,a4a20a50-abfd-4ee8-9741-4e22eaf821d0,f604439a-66bc-4ad5-a35c-c4e5368dd7b0,e163a007-2139-427d-b878-58eaa2a5c81b,54e3b8d8-0653-4466-a05a-712a0069dcec
3,8cea2d02-9bed-4519-b33a-ba712767cf45,03cbfc57-67fa-4f76-aad8-d64fb28002f6,e87902c3-c62e-48b6-9b1a-851f9dc164ae,2b8f7076-743d-4208-a84e-9d214d36705d
4,6b91fcbb-b8bf-4ca7-90f4-1de83c0f5257,b541a36d-2e0b-4eda-ac96-eab938b5249c,81263b9c-871f-45a1-be63-93425afc20d3,d9e6f46c-45b3-4c15-acc4-767f9e382abd


In [7]:
bookings.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   user_id                    150000 non-null  object
 1   booked_event_id            150000 non-null  object
 2   booked_event_organizer_id  150000 non-null  object
 3   booking_id                 150000 non-null  object
dtypes: object(4)
memory usage: 4.6+ MB


In [8]:
comments.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245397 entries, 0 to 245396
Data columns (total 4 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   user_id                       245397 non-null  object
 1   commented_event_id            245397 non-null  object
 2   commented_event_organizer_id  245397 non-null  object
 3   comment_id                    245397 non-null  object
dtypes: object(4)
memory usage: 7.5+ MB


In [9]:
likes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145397 entries, 0 to 145396
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   user_id                   145397 non-null  object
 1   liked_event_id            145397 non-null  object
 2   liked_event_organizer_id  145397 non-null  object
 3   like_id                   145397 non-null  object
dtypes: object(4)
memory usage: 4.4+ MB


In [10]:
# Standardize column names across all datasets
bookings = bookings.rename(columns={
    'booked_event_id': 'event_id',
    'booked_event_organizer_id': 'organizer_id',
    'booking_id': 'id'
})

comments = comments.rename(columns={
    'commented_event_id': 'event_id',
    'commented_event_organizer_id': 'organizer_id',
    'comment_id': 'id'
})

likes = likes.rename(columns={
    'liked_event_id': 'event_id',
    'liked_event_organizer_id': 'organizer_id',
    'like_id': 'id'
})

# Create source-specific DataFrames with indicator columns
bookings_df = bookings[['user_id', 'event_id', 'organizer_id']].copy()
bookings_df['is_booked'] = 1
bookings_df['num_bookings'] = 1

comments_df = comments[['user_id', 'event_id', 'organizer_id']].copy()
comments_df['is_commented'] = 1
comments_df['num_comments'] = 1

likes_df = likes[['user_id', 'event_id', 'organizer_id']].copy()
likes_df['is_liked'] = 1
likes_df['num_likes'] = 1

# Get all unique user-event pairs
unique_pairs = pd.concat([
    bookings[['user_id', 'event_id']],
    comments[['user_id', 'event_id']],
    likes[['user_id', 'event_id']]
]).drop_duplicates()

# Randomly select 100,000 unique pairs if we have more than that
if len(unique_pairs) > 200000:
    unique_pairs = unique_pairs.sample(n=200000, random_state=42)

# Get consistent organizer_id mapping (most frequent for each event)
organizer_mapping = pd.concat([
    bookings[['event_id', 'organizer_id']],
    comments[['event_id', 'organizer_id']],
    likes[['event_id', 'organizer_id']]
]).groupby('event_id')['organizer_id'].agg(lambda x: x.mode()[0]).to_dict()

# Merge all interaction types
result = unique_pairs.copy()

# Merge booking data
result = result.merge(
    bookings_df.groupby(['user_id', 'event_id']).agg({
        'is_booked': 'max',
        'num_bookings': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge comment data
result = result.merge(
    comments_df.groupby(['user_id', 'event_id']).agg({
        'is_commented': 'max',
        'num_comments': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Merge like data
result = result.merge(
    likes_df.groupby(['user_id', 'event_id']).agg({
        'is_liked': 'max',
        'num_likes': 'sum',
        'organizer_id': 'first'
    }).reset_index(),
    on=['user_id', 'event_id'],
    how='left'
)

# Fill NA values with 0 for indicator and count columns
for col in ['is_booked', 'num_bookings', 'is_commented', 'num_comments', 'is_liked', 'num_likes']:
    result[col] = result[col].fillna(0).astype(int)

# Consolidate organizer_id (take first non-NA value)
result['organizer_id'] = (result['organizer_id_x']
                         .fillna(result['organizer_id_y'])
                         .fillna(result['organizer_id']))

# If still missing, use our mapping
result['organizer_id'] = result['organizer_id'].fillna(result['event_id'].map(organizer_mapping))

# Select final columns
final_data = result[[
    'user_id', 
    'event_id', 
    'is_booked',
    'num_bookings',
    'is_liked',
    'num_likes',
    'is_commented', 
    'num_comments', 
    'organizer_id'
]]

# Ensure we have exactly 100,000 records (in case some merges added duplicates)
final_data = final_data.drop_duplicates().sample(n=min(200000, len(final_data)), random_state=42)

# Save the merged data
final_data.to_csv("artifacts/data_ingestion/generated_data/merged_interactions.csv", index=False)

print("Merged data saved successfully!")
print(f"Final dataset shape: {final_data.shape}")
print(final_data.head())

Merged data saved successfully!
Final dataset shape: (200000, 9)
                                     user_id  \
119737  e10fd3ff-0b93-4214-ac6f-7d323dac5562   
72272   1607fa73-d544-4a6b-acc7-1515f6f8cfc8   
158154  5a2cb656-2dfe-449d-9c89-91e1f7578f4a   
65426   e42cdecc-8eed-4e1f-91b0-772e03c1df44   
30074   92073ec5-50f6-47fa-9d8c-cb4dbd7028e9   

                                    event_id  is_booked  num_bookings  \
119737  4c4ac787-6c1b-4cc0-98a4-5a28dd05a414          0             0   
72272   7b347f14-3dcf-40b9-8be3-7d032cacfb57          0             0   
158154  9a872252-89e6-442a-b707-03b230559366          1             4   
65426   23ca6187-34eb-4a99-a1a7-b1660b58b6a7          0             0   
30074   d7cdd08e-db62-4295-9246-680a72d28190          0             0   

        is_liked  num_likes  is_commented  num_comments  \
119737         0          0             1             1   
72272          0          0             1             3   
158154         0          0   